## TYTAN tutorial おすすめ2（温度計パズル）

最終更新：2023年6月25日 by ビネクラ安田

取り組み方

*   Google Colabで取り組む場合：ファイル＞ドライブにコピーを保存　<font color="red">※このファイルを直接編集しても保存されません</font>
*   Jupyter Notebookに移す場合：ファイル＞ダウンロード＞.ipynbをダウンロード

参考リンク１

*   [TYTANチュートリアル一覧](https://github.com/tytansdk/tytan_tutorial)
*   [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

出展

*   [量子アニーリング（QUBO）で温度計パズル（Thermometers）を解く](https://vigne-cla.com/21-16/)

### 問題
温度計パズル（Thermometers）と呼ばれるパズル。

各行、各列とも数字の数だけ赤マスとして塗るが、温度計なので球部（水銀溜まり）から順にしか塗られないという制限がある。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_1-300x300.png" width = 20%>
</div>

答えはこちら。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_2-300x300.png" width = 20%>
</div>

ブラウザ上で遊んでみると良い → [Puzzle Team](https://www.puzzle-thermometers.com/)

### このQUBO設定を使おう

<font color="red">「n個の量子ビットからm個を1にする」</font>

例）3個の量子ビットから2個を1にする
```
H = (q1 + q2 + q3 - 2)**2
```

<font color="red">「2個の量子ビットを降順にする」</font>

例）q0 ≧ q1 にする（＝[0, 1] になったらペナルティを与える）
```
H = (1 - q0) * q1
```

<font color="red">「すべての量子ビットを降順にする」</font>

例）q0 ≧ q1 ≧ q2 にする
```
H = (1 - q0) * q1
H += (1 - q1) * q2

```

その他の条件式も気になる方は → [量子アニーリングのQUBOで設定可能な条件式まとめ（保存版）](https://vigne-cla.com/21-12/)

> インデントされたブロック



### 制約条件

16個の量子ビットを各マスに対応させる。結果が１なら色を塗ることとする。

まず、各行、各列とも指定の個数だけ１になるように設定。基本の「ｎ個の量子ビットからｍ個を１にする」を使う。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_3r.png" width = 45%>
</div>

次に、各温度計に着目して、球部から降順になるように設定。ここで言う降順とは q0 ≧ q1 ≧ q2 の意味で、[1, 1, 0] のように途中から0に切り替わる形だけでなく、[0, 0, 0]、[1, 1, 1] のようにずっと等しくてもOK。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_4.png" width = 45%>
</div>

## コード

In [ ]:
!pip install git+https://github.com/tytansdk/tytan

In [8]:
from tytan import *
import numpy as np

#量子ビットを用意する
q00 = symbols('q00')
q01 = symbols('q01')
q02 = symbols('q02')
q03 = symbols('q03')
q04 = symbols('q04')
q05 = symbols('q05')
q06 = symbols('q06')
q07 = symbols('q07')
q08 = symbols('q08')
q09 = symbols('q09')
q10 = symbols('q10')
q11 = symbols('q11')
q12 = symbols('q12')
q13 = symbols('q13')
q14 = symbols('q14')
q15 = symbols('q15')

#各行、「4個から指定の個数だけ1になる」
H = 0
H += (q00 + q01 + q02 + q03 - 2)**2
H += (q04 + q05 + q06 + q07 - 1)**2
H += (q08 + q09 + q10 + q11 - 3)**2
H += (q12 + q13 + q14 + q15 - 1)**2

#各列、「4個から指定の個数だけ1になる」
H += (q00 + q04 + q08 + q12 - 3)**2
H += (q01 + q05 + q09 + q13 - 1)**2
H += (q02 + q06 + q10 + q14 - 1)**2
H += (q03 + q07 + q11 + q15 - 2)**2

#各温度計、球部から降順になる
H += (1 - q08) * q04
H += (1 - q04) * q00 #8→4→0の連鎖

H += (1 - q05) * q01

H += (1 - q03) * q02

H += (1 - q07) * q06

H += (1 - q11) * q10
H += (1 - q10) * q09 #11→10→9の連鎖

H += (1 - q13) * q12

H += (1 - q15) * q14


#コンパイル
qubo, offset = Compile(H).get_qubo()

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(r)
    print(np.array(list(r[0].values())).reshape(4, 4))

[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 1, 'q05': 0, 'q06': 0, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 1, 'q11': 1, 'q12': 0, 'q13': 1, 'q14': 0, 'q15': 0}, -30.0, 31]
[[1 0 0 1]
 [1 0 0 0]
 [1 0 1 1]
 [0 1 0 0]]
[{'q00': 1, 'q01': 0, 'q02': 0, 'q03': 1, 'q04': 1, 'q05': 0, 'q06': 0, 'q07': 0, 'q08': 1, 'q09': 1, 'q10': 1, 'q11': 0, 'q12': 0, 'q13': 0, 'q14': 0, 'q15': 1}, -29.0, 33]
[[1 0 0 1]
 [1 0 0 0]
 [1 1 1 0]
 [0 0 0 1]]
[{'q00': 1, 'q01': 1, 'q02': 0, 'q03': 0, 'q04': 1, 'q05': 0, 'q06': 0, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 1, 'q11': 1, 'q12': 0, 'q13': 0, 'q14': 0, 'q15': 1}, -29.0, 13]
[[1 1 0 0]
 [1 0 0 0]
 [1 0 1 1]
 [0 0 0 1]]
[{'q00': 0, 'q01': 0, 'q02': 1, 'q03': 1, 'q04': 1, 'q05': 0, 'q06': 0, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 0, 'q11': 1, 'q12': 0, 'q13': 1, 'q14': 0, 'q15': 0}, -28.0, 3]
[[0 0 1 1]
 [1 0 0 0]
 [1 0 0 1]
 [0 1 0 0]]
[{'q00': 0, 'q01': 0, 'q02': 1, 'q03': 1, 'q04': 1, 'q05': 0, 'q06': 0, 'q07': 0, 'q08': 1, 'q09': 0, 'q10': 0, 'q11': 1, 'q1

一番目の解は模範解答の通りである。

<div align="center">
<img src="https://vigne-cla.com/wp-content/uploads/2023/05/21-16_2-300x300.png" width = 20%>
</div>

## おまけ：TYTANの便利関数を使用したコード

ここではfor文で設定することを見据えてsymbols_list()関数を使用した例を紹介する。

symbols_define()関数やsymbols_list()関数を使用する場合、量子ビットの添字は0埋めされず、結果の配列が添字のアルファベットソートであることに注意。Auto_array().get_ndarray()関数で自然順ソートされるため、これらをセットで用いると良い。

詳しくはドキュメント → [TYTANドキュメント](https://github.com/tytansdk/tytan/blob/main/document%20.md)

In [2]:
from tytan import *
import numpy as np

#量子ビットを用意する
q = symbols_list([4, 4], 'q{}_{}')
print(q)

#各行、「4個から指定の個数だけ1になる」
H = 0
H += (q[0][0] + q[0][1] + q[0][2] + q[0][3] - 2)**2
H += (q[1][0] + q[1][1] + q[1][2] + q[1][3] - 1)**2
H += (q[2][0] + q[2][1] + q[2][2] + q[2][3] - 3)**2
H += (q[3][0] + q[3][1] + q[3][2] + q[3][3] - 1)**2

#各列、「4個から指定の個数だけ1になる」
H += (q[0][0] + q[1][0] + q[2][0] + q[3][0] - 3)**2
H += (q[0][1] + q[1][1] + q[2][1] + q[3][1] - 1)**2
H += (q[0][2] + q[1][2] + q[2][2] + q[3][2] - 1)**2
H += (q[0][3] + q[1][3] + q[2][3] + q[3][3] - 2)**2

#各温度計、球部から降順になる
H += (1 - q[2][0]) * q[1][0]
H += (1 - q[1][0]) * q[0][0] #8→4→0の連鎖

H += (1 - q[1][1]) * q[0][1]

H += (1 - q[0][3]) * q[0][2]

H += (1 - q[1][3]) * q[1][2]

H += (1 - q[2][3]) * q[2][2]
H += (1 - q[2][2]) * q[2][1] #11→10→9の連鎖

H += (1 - q[3][1]) * q[3][0]

H += (1 - q[3][3]) * q[3][2]


#コンパイル
qubo, offset = Compile(H).get_qubo()
print(f'offset\n{offset}')

#サンプラー選択
solver = sampler.SASampler()

#サンプリング
result = solver.run(qubo)

#上位5件
for r in result[:5]:
    print(f'Energy {r[1]}, Occurrence {r[2]}')
    arr, subs = Auto_array(r[0]).get_ndarray('q{}_{}')
    print(arr)

[[q0_0 q0_1 q0_2 q0_3]
 [q1_0 q1_1 q1_2 q1_3]
 [q2_0 q2_1 q2_2 q2_3]
 [q3_0 q3_1 q3_2 q3_3]]
offset
30
Energy -30.0, Occurrence 56
[[1 0 0 1]
 [1 0 0 0]
 [1 0 1 1]
 [0 1 0 0]]
Energy -29.0, Occurrence 15
[[1 0 0 1]
 [1 0 0 0]
 [1 1 1 0]
 [0 0 0 1]]
Energy -29.0, Occurrence 18
[[1 1 0 0]
 [1 0 0 0]
 [1 0 1 1]
 [0 0 0 1]]
Energy -28.0, Occurrence 2
[[0 0 1 1]
 [1 0 0 0]
 [1 0 0 1]
 [1 1 0 0]]
Energy -28.0, Occurrence 2
[[0 0 1 1]
 [1 0 0 0]
 [1 0 1 1]
 [1 1 0 0]]
